<a href="https://colab.research.google.com/github/hollynroper/URS-22-23/blob/main/IST_and_AMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
from math import sqrt

# Prepping for execution

The two fuctions in the next cell are the implementation of these two equations.

The Onsager Correction term:
\begin{equation}   \mu^t = \frac{1}{n}\mathbf{z}^{t-1}\sum\eta^{'}(r_j^{t-1};\tau_{t-1})
\end{equation}

The Eta function: \begin{equation}    \eta(u;T) = 
    \left\{
    \begin{array}{lr}
        u - T &\text{if } u \ge T \\
        u + T &\text{if } u \le -T \\
        0 \text{ else}&
    \end{array}
    \right\} \end{equation}

Also included are system parameters. Where n is the number of measurements, N is the number of total users, and k is the number of active users. λ and s are thresholding parameters.

In [ ]:
def onsager(z, r, tau):
    return (z/n) * np.sum(eta(r, tau) != 0)

def eta(u, T):
    return (u - T)*(u >= T) + (u + T)*(u <= -T)

# initial parameters
n = 270
N = 1024
k = 40
s = 0.2
lambda_ = 0.1

A is the sensing matrix. It conists of Gaussian random variables with variance equal to 1/n.

w is the noise vector.

x is the messge vector that we will try to recover.

y is the compressed signal received by the station.

In [ ]:
# create sensing matrix A
A = np.sqrt(1/n)*np.random.randn(n, N)

# finding thresholding parameter
_, Lambda, _ = np.linalg.svd(A)
L = np.max(Lambda) + 1

# creating noise
SNRs = np.array([-12,-10, -5, 1, 5, 10, 20])
SNRs = (10**(SNRs/10))*n
sigma = 1/(np.sqrt(SNRs))
w = sigma[2]*np.random.randn(n).reshape(-1,1)

# create k sparse x vector
x = np.zeros((N, 1))
idx_nonzero_entries = np.random.permutation(N)[0:k]
x[idx_nonzero_entries] = 1

y = A @ x + w

# MSE vs Iterations

In [ ]:
num_iterations = 15

**Iterative Soft Tresholding**

Choosing alpha

In [ ]:
num_iterations = 15
alphas = [.05, .1, .15, .2, .25, .3, .35]
mse_aL = []
mse = np.zeros(num_iterations)

for a in alphas:
    xHt = np.zeros(x.shape)
    for idx_iter in range(num_iterations):
        z = y - A @ xHt
        r = xHt + s * A.T @ z
        xHt = eta(r, a/L)
        # find the current error
        mse[idx_iter] = (1/N) * np.sum((x - xHt)**2)
    mse_aL.append(mse)
    mse = np.zeros(num_iterations)

# divide into respective arrays to plot
mse_05 = mse_aL[0]
mse_1 = mse_aL[1]
mse_15 = mse_aL[2]
mse_2 = mse_aL[3]
mse_25 = mse_aL[4]
mse_3 = mse_aL[5]
mse_35 = mse_aL[6]

# plotting the results
plt.figure()
plt.plot(range(1, num_iterations+1), mse_05, label="alpha=.05")
plt.plot(range(1, num_iterations+1), mse_1, label="alpha=.1")
plt.plot(range(1, num_iterations+1), mse_15, label="alpha=.15")
plt.plot(range(1, num_iterations+1), mse_2, label="alpha=.2")
plt.plot(range(1, num_iterations+1), mse_25, label="alpha=.25")
plt.plot(range(1, num_iterations+1), mse_3, label="alpha=.3")
plt.plot(range(1, num_iterations+1), mse_35, label="alpha=.35")
plt.title("MSE vs iterations")
plt.xlabel("Number of iterations")
plt.ylabel("MSE")
plt.legend()
plt.grid(True, which='both')
plt.show()

From the above results, we decided to use alpha = .2

In [ ]:
# IST
alpha = .2
ist_mse = np.zeros(num_iterations)
xHt = np.zeros(x.shape)

for idx_iter in range(num_iterations):
    z = y - A @ xHt
    r = xHt + s * A.T @ z  ### s -- what do I do with it
    xHt = eta(r, alpha/L)
    # find the current error
    ist_mse[idx_iter] = (1/N) * np.sum((x - xHt)**2)

**Approximate Message Passing**

In [ ]:
#AMP
amp_mse = np.zeros(num_iterations)
z = np.zeros(y.shape)
xHt = np.zeros(x.shape)
r = 0
tau = 0
ons = 0

for idx_iter in range(num_iterations):
    ons = onsager(z, r, tau)
    z = y - A @ xHt + ons
    tau = np.sqrt(1/n) * np.linalg.norm(z)
    r = xHt + A.T @ z
    xHt = eta(r, tau)

    # find the current error
    amp_mse[idx_iter] = (1/N) * np.sum((x - xHt)**2)

In [ ]:
# MSE vs itertations. Linear Scale
plt.figure()
plt.plot(range(1,num_iterations+1), ist_mse, label="IST", color="green")
plt.plot(range(1,num_iterations+1), amp_mse, label="AMP", color="darkorange")
plt.title("MSE vs iterations (Linear scale)")
plt.xlabel("Number of iterations")
plt.ylabel("MSE")
plt.legend()
plt.grid(True, which='both')
plt.show()

# MSE vs itertations. Log Scale
plt.figure()
plt.semilogy(range(1,num_iterations+1), ist_mse, label="IST", color="green")
plt.semilogy(range(1,num_iterations+1), amp_mse, label="AMP", color="darkorange")
plt.title("MSE vs iterations (Log scale)")
plt.xlabel("Number of iterations")
plt.ylabel("MSE")
plt.legend()
plt.grid(True, which='both')
plt.show()

# MSE vs SNR

In [ ]:
SNRdB = np.array([-12,-10, -5, 1, 5, 10, 20])
SNRs = (10**(SNRdB/10))*n
sigma = 1/(np.sqrt(SNRs))

**Iterative Soft Thresholding**
!!! Still need to remove 's' !!!!

In [ ]:
num_iterations = 15
cnt = 0
ist_mse_snr = np.zeros(len(sigma))

for sig in sigma:
    mse = 0
    for i in range(100):
        # creating new noise
        w = sig*np.random.randn(n).reshape(-1,1)
        x = np.zeros((N, 1))
        idx_nonzero_entries = np.random.permutation(N)[0:k]
        x[idx_nonzero_entries] = 1
        y = A @ x + w

        xHt = np.zeros(x.shape)

        # IST
        for idx_iter in range(num_iterations):
            z = y - A @ xHt
            r = xHt + s * A.T @ z   ## this 's' can't stick around
            xHt = eta(r, alpha/L)
        mse += (1/N) * np.sum((x - xHt)**2)
        # find the current error
    ist_mse_snr[cnt] = (1/100) * mse
    cnt += 1

**Approximate Message Passing**

In [ ]:
num_iterations = 15
cnt = 0
amp_mse_snr = np.zeros(len(sigma))
mse = 0

for sig in sigma:
    mse = 0
    for i in range(100):    
        # creating new noise
        w = sig*np.random.randn(n).reshape(-1,1)
        x = np.zeros((N, 1))
        idx_nonzero_entries = np.random.permutation(N)[0:k]
        x[idx_nonzero_entries] = 1
        y = A @ x + w

        # prepping for AMP
        z = np.zeros(y.shape)
        r = 0
        tau = 0
        ons = 0
        xHt = np.zeros(x.shape)

        # AMP
        for idx_iter in range(num_iterations):
            ons = onsager(z, r, tau)
            z = y - A @ xHt + ons
            tau = np.sqrt(1/n) * np.linalg.norm(z)
            r = xHt + A.T @ z
            xHt = eta(r, tau)
        mse += (1/N) * np.sum((x - xHt)**2)
    amp_mse_snr[cnt] = (1/100) * mse
    cnt += 1

In [ ]:
# MSE vs SNR. 15 iterations. Linear Scale
plt.figure()
plt.plot(SNRdB, ist_mse_snr, label="IST", color="green")
plt.plot(SNRdB, amp_mse_snr, label="AMP", color="orange")
plt.title("MSE vs SNR")
plt.xlabel("SNR (dB)")
plt.ylabel("MSE")
plt.legend()
plt.grid(True, which='both')
plt.show()

# MSE vs SNR. 15 iterations. Log Scale
plt.figure()
plt.semilogy(SNRdB, ist_mse_snr, label="IST", color="green")
plt.semilogy(SNRdB, amp_mse_snr, label="AMP", color="orange")
plt.title("MSE vs SNR")
plt.xlabel("SNR (dB)")
plt.ylabel("MSE")
plt.legend()
plt.grid(True, which='both')
plt.show()